# Tourism Data Cleaning

This notebook is designed to load and clean the tourism data from the [World Tourism Organization](https://www.unwto.org/tourism-statistics/key-tourism-statistics). The data `unwto-all-data-download_2022.xlsx` is downloaded from the website and saved in the `raw` folder. The cleaned data is saved in the `export` folder.

In [12]:
import numpy as np
import pandas as pd
from IPython.display import display
pd.options.mode.copy_on_write = True 

# Load the Excel file
file_path = "../data/raw/unwto-all-data-download_2022.xlsx"
xls = pd.ExcelFile(file_path)

# List sheet names to verify
print("Available Sheets:")
print(pd.DataFrame(xls.sheet_names, columns=["Sheet Names"]))

Available Sheets:
                       Sheet Names
0                            Index
1         Inbound Tourism-Arrivals
2          Inbound Tourism-Regions
3          Inbound Tourism-Purpose
4        Inbound Tourism-Transport
5    Inbound Tourism-Accommodation
6      Inbound Tourism-Expenditure
7           Domestic Tourism-Trips
8   Domestic Tourism-Accommodation
9      Outbound Tourism-Departures
10    Outbound Tourism-Expenditure
11              Tourism Industries
12                      Employment


## Identifying the sheets

The data is stored in multiple sheets. We will identify the sheet that contains the data we are interested in.


In [13]:
# Load the relevant sheets, skipping the first two rows
df_arrivals = xls.parse(" Inbound Tourism-Arrivals", skiprows=2)
df_transport = xls.parse("Inbound Tourism-Transport", skiprows=2)
df_regions = xls.parse("Inbound Tourism-Regions", skiprows=2)

# Display first few rows of each sheet to understand structure
print("\nInbound Tourism-Arrivals Sample:")
display(df_arrivals.head())

print("\nInbound Tourism-Transport Sample:")
display(df_transport.head())

print("\nInbound Tourism-Regions Sample:")
display(df_regions.head())


Inbound Tourism-Arrivals Sample:


,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 39
0,4.0,0.0,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Arrivals,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.1,4-1.1,NaN,NaN,Total arrivals,NaN,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,1.2,4-1.2,NaN,NaN,NaN,Overnights visitors (tourists),NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,1.3,4-1.3,NaN,NaN,NaN,Same-day visitors (excursionists),NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN



Inbound Tourism-Transport Sample:


,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 39
0,4.0,0.00,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Arrivals by mode of transport,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.19,4-1.19,NaN,NaN,Total,NaN,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,1.20,4-1.20,NaN,NaN,NaN,Air,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,1.21,4-1.21,NaN,NaN,NaN,Water,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN



Inbound Tourism-Regions Sample:


,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 39
0,4.0,0.0,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Arrivals by region,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.5,4-1.5,NaN,NaN,Total,NaN,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,1.6,4-1.6,NaN,NaN,NaN,Africa,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,1.7,4-1.7,NaN,NaN,NaN,Americas,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN


## Filtering the data

We will filter the data to include only the columns that are relevant to our analysis. We will also filter the data to include only the rows that contain data for the countries we are interested in.

In [22]:
# Inspect the column names of the DataFrames
print("Columns in df_arrivals:", df_arrivals.columns)
print("Columns in df_transport:", df_transport.columns)
print("Columns in df_regions:", df_regions.columns)

# Define the correct columns of interest based on the actual column names
columns_of_interest_arrivals = ["Basic data and indicators"] + [str(year) for year in range(1995, 2023)]

# Adjust columns_of_interest to match the actual column names
columns_of_interest_arrivals = [col for col in columns_of_interest_arrivals if col in df_arrivals.columns]

# Define the correct columns of interest based on the actual column names
columns_of_interest_transport = ["Basic data and indicators"] + [str(year) for year in range(1995, 2023)]

# Adjust columns_of_interest to match the actual column names
columns_of_interest_transport = [col for col in columns_of_interest_transport if col in df_arrivals.columns]

# Filter the datasets to only include relevant columns
df_arrivals_filtered = df_arrivals[columns_of_interest_arrivals]
df_transport_filtered = df_transport[columns_of_interest_transport]
# df_regions_filtered = df_regions[columns_of_interest]

Columns in df_arrivals: Index(['Basic data and indicators',                        1995,
                              1996,                        1997,
                              1998,                        1999,
                              2000,                        2001,
                              2002,                        2003,
                              2004,                        2005,
                              2006,                        2007,
                              2008,                        2009,
                              2010,                        2011,
                              2012,                        2013,
                              2014,                        2015,
                              2016,                        2017,
                              2018,                        2019,
                              2020,                        2021,
                              2022],
      dtype='object')
Columns

In [16]:
display(df_arrivals)

,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 39
0,4.0,0.0,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Arrivals,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.1,4-1.1,NaN,NaN,Total arrivals,NaN,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,1.2,4-1.2,NaN,NaN,NaN,Overnights visitors (tourists),NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,1.3,4-1.3,NaN,NaN,NaN,Same-day visitors (excursionists),NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,NaN,NaN,NaN,VF,Arrivals of non-resident visitors at national ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,NaN,NaN,NaN,THS,Arrivals of non-resident tourists in hotels an...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343,NaN,NaN,NaN,TCE,Arrivals of non-resident tourists in all types...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1344,NaN,NaN,NaN,..,Data not available,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
display(df_transport)

,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 39
0,4.0,0.00,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Arrivals by mode of transport,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.19,4-1.19,NaN,NaN,Total,NaN,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,1.20,4-1.20,NaN,NaN,NaN,Air,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,1.21,4-1.21,NaN,NaN,NaN,Water,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,NaN,NaN,NaN,VF,Arrivals of non-resident visitors at national ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,NaN,NaN,NaN,THS,Arrivals of non-resident tourists in hotels an...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343,NaN,NaN,NaN,TCE,Arrivals of non-resident tourists in all types...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1344,NaN,NaN,NaN,..,Data not available,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
display(df_regions)

,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 39
0,4.0,0.0,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Arrivals by region,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.5,4-1.5,NaN,NaN,Total,NaN,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,1.6,4-1.6,NaN,NaN,NaN,Africa,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,1.7,4-1.7,NaN,NaN,NaN,Americas,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,NaN,NaN,NaN,VF,Arrivals of non-resident visitors at national ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2457,NaN,NaN,NaN,THS,Arrivals of non-resident tourists in hotels an...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2458,NaN,NaN,NaN,TCE,Arrivals of non-resident tourists in all types...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2459,NaN,NaN,NaN,..,Data not available,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Drop the specified columns
columns_to_drop = ["C.", "S.", "C. & S.", "Unnamed: 4", "Unnamed: 5", "Unnamed: 6","Unnamed: 7", "Unnamed: 39", "Units", "Notes", 'Series']
df_arrivals.drop(columns=columns_to_drop, inplace=True)
df_transport.drop(columns=columns_to_drop, inplace=True)
df_regions


,Basic data and indicators,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
3,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,VF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,THS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343,TCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1344,..,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Display the updated dataframe
display(df_arrivals)

,Basic data and indicators,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
3,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,VF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,THS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343,TCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1344,..,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
display(df_transport)

,C.,S.,C. & S.,Basic data and indicators,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Units,Notes,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 39
0,4.0,0.00,4-0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Arrivals by mode of transport,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,1.19,4-1.19,NaN,NaN,Total,NaN,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
3,4.0,1.20,4-1.20,NaN,NaN,NaN,Air,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
4,4.0,1.21,4-1.21,NaN,NaN,NaN,Water,NaN,Thousands,NaN,...,..,..,..,..,..,..,..,..,..,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,NaN,NaN,NaN,VF,Arrivals of non-resident visitors at national ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,NaN,NaN,NaN,THS,Arrivals of non-resident tourists in hotels an...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343,NaN,NaN,NaN,TCE,Arrivals of non-resident tourists in all types...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1344,NaN,NaN,NaN,..,Data not available,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



## Cleaning the data

The data is cleaned by removing unnecessary rows and columns, renaming columns, and converting the data to the appropriate data types.

In [18]:
# Forward fill the missing values in the first column
df_arrivals['Basic data and indicators'].fillna(method='ffill', inplace=True)

# Display the updated dataframe
display(df_arrivals)

/var/folders/0y/g2tm105x5fj4b33ds4k_rtb00000gn/T/ipykernel_27451/3972088124.py:2: ChainedAssignmentError: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
When using the Copy-on-Write mode, such inplace method never works to update the original DataFrame or Series, because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' instead, to perform the operation inplace on the original object.


  df_arrivals['Basic data and indicators'].fillna(method='ffill', inplace=True)
/var/folders/0y/g2tm105x5fj4b33ds4k_rtb00000gn/T/ipykernel_27451/3972088124.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_arrivals['Basic data and indicators'].fillna(method='ffill', inplace=True)


,Basic data and indicators,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,AFGHANISTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
3,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,VF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1342,THS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1343,TCE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1344,..,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Group the dataframe by every 6 rows and select rows from position 2 to 4 within each group
df_arrivals_filtered = df_arrivals.groupby(df_arrivals.index // 6).apply(lambda x: x.iloc[2:4]).reset_index(drop=True)

# Display the collected rows
display(df_arrivals_filtered)

,Basic data and indicators,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,NaN,..,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,NaN,304,287,119,184,371,317,354,470,557,...,3256,3673,4131,4736,5118,5927,6406,2658,5689,7543.8
3,NaN,..,..,..,..,..,..,..,..,..,...,2857,3341,3784,4070,4643,5340,6128,2604,5515,7104.7
4,NaN,520,605,635,678,749,866,901,988,1166,...,2733,2301,1710,2039,2451,2657,2371,591,125,1398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,NaN,163,264,341,362,404,457,492,565,413,...,915,947,932,956,1009,1072,1266,502,554,..
444,NaN,1416,1597,1336,2090,2250,1967,2217,2041,2256,...,1833,1880,2057,2168,2423,2580,2294,639,381,1044
445,NaN,1363,1577,1281,1986,2101,1868,2068,..,..,...,..,..,..,..,..,..,..,..,..,..
446,TF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Convert the dataframe to numeric, forcing errors to NaN, excluding the "Basic data and indicators" column
df_arrivals_filtered_numeric = df_arrivals_filtered.drop(columns=["Basic data and indicators"]).apply(pd.to_numeric, errors='coerce')

# Compare every two rows and replace the columns with the max value
df_max_values = df_arrivals_filtered_numeric.groupby(np.arange(len(df_arrivals_filtered_numeric)) // 2).max()

# Add the "Basic data and indicators" column back to the dataframe
df_max_values["Basic data and indicators"] = df_arrivals_filtered["Basic data and indicators"].iloc[::2].values

# Reorder columns to place "Basic data and indicators" at the beginning
df_max_values = df_max_values[["Basic data and indicators"] + df_max_values.columns[:-1].tolist()]

# Display the new dataframe
display(df_max_values)

,Basic data and indicators,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,304.0,287.0,119.0,184.0,371.0,317.0,354.0,470.0,557.0,...,3256.0,3673.0,4131.0,4736.0,5118.0,5927.0,6406.0,2658.0,5689.0,7543.8
2,NaN,520.0,605.0,635.0,678.0,749.0,866.0,901.0,988.0,1166.0,...,2733.0,2301.0,1710.0,2039.0,2451.0,2657.0,2371.0,591.0,125.0,1398.0
3,NaN,34.0,35.0,26.0,36.0,41.0,44.0,36.0,NaN,NaN,...,49.3,51.6,47.1,38.3,42.3,51.8,58.6,0.9,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,9422.0,10991.0,11351.0,11507.0,11601.0,...,7676.0,7797.0,7850.0,8025.0,8152.0,8328.0,8235.0,5207.0,5422.0,8426.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,NaN,1351.0,1607.0,1716.0,1520.0,1782.0,2140.0,2330.0,2628.0,2429.0,...,7572.0,7874.0,7944.0,10013.0,12922.0,15498.0,18009.0,3837.0,157.0,3661.0
220,NaN,61.0,74.0,80.0,88.0,58.0,73.0,76.0,98.0,155.0,...,1323.0,1218.0,398.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,NaN,163.0,264.0,341.0,362.0,404.0,457.0,492.0,565.0,413.0,...,915.0,947.0,932.0,956.0,1009.0,1072.0,1266.0,502.0,554.0,NaN
222,NaN,1416.0,1597.0,1336.0,2090.0,2250.0,1967.0,2217.0,2041.0,2256.0,...,1833.0,1880.0,2057.0,2168.0,2423.0,2580.0,2294.0,639.0,381.0,1044.0


In [21]:
# Drop rows where all columns (except 'Basic data and indicators') have no numerical values
df_final_Inbound_Tourism_Arrivals = df_max_values.dropna(subset=df_max_values.columns[1:], how='all')

# Replace missing values with 0
df_final_Inbound_Tourism_Arrivals.fillna(0, inplace=True)

# Display the final DataFrame
display(df_final_Inbound_Tourism_Arrivals)

/var/folders/0y/g2tm105x5fj4b33ds4k_rtb00000gn/T/ipykernel_27451/117615975.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final_Inbound_Tourism_Arrivals.fillna(0, inplace=True)


,Basic data and indicators,1995,1996,1997,1998,1999,2000,2001,2002,2003,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
1,0,304.0,287.0,119.0,184.0,371.0,317.0,354.0,470.0,557.0,...,3256.0,3673.0,4131.0,4736.0,5118.0,5927.0,6406.0,2658.0,5689.0,7543.8
2,0,520.0,605.0,635.0,678.0,749.0,866.0,901.0,988.0,1166.0,...,2733.0,2301.0,1710.0,2039.0,2451.0,2657.0,2371.0,591.0,125.0,1398.0
3,0,34.0,35.0,26.0,36.0,41.0,44.0,36.0,0.0,0.0,...,49.3,51.6,47.1,38.3,42.3,51.8,58.6,0.9,0.0,0.0
4,0,0.0,0.0,0.0,0.0,9422.0,10991.0,11351.0,11507.0,11601.0,...,7676.0,7797.0,7850.0,8025.0,8152.0,8328.0,8235.0,5207.0,5422.0,8426.7
5,0,9.0,21.0,45.0,52.0,45.0,51.0,67.0,91.0,107.0,...,650.0,595.0,592.0,397.0,261.0,218.0,218.0,64.0,64.0,130.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
218,0,879.0,960.0,933.0,813.0,702.0,602.0,792.0,590.0,435.0,...,1085.0,967.0,882.0,681.0,429.0,0.0,0.0,0.0,0.0,0.0
219,0,1351.0,1607.0,1716.0,1520.0,1782.0,2140.0,2330.0,2628.0,2429.0,...,7572.0,7874.0,7944.0,10013.0,12922.0,15498.0,18009.0,3837.0,157.0,3661.0
220,0,61.0,74.0,80.0,88.0,58.0,73.0,76.0,98.0,155.0,...,1323.0,1218.0,398.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
221,0,163.0,264.0,341.0,362.0,404.0,457.0,492.0,565.0,413.0,...,915.0,947.0,932.0,956.0,1009.0,1072.0,1266.0,502.0,554.0,0.0
